**Loading file from Google Drive**

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
downloaded = drive.CreateFile({'id':"1XHJafllpgJweuFIfZ0FX79G2kY1M91TJ"})   
downloaded.GetContentFile('Data_Raw_signals.pkl') 

**Proprocess then Split Data (train/val)**

In [0]:
import os
import glob
import pickle as pkl
import numpy as np
import warnings
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from keras.utils import np_utils

from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv1D, Input, MaxPooling1D, TimeDistributed, LSTM
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.regularizers import l2
# from keras import backend as K
from tensorflow.keras import backend as K
K.clear_session()

warnings.filterwarnings("ignore")

raw_signal_dir = 'Data_Raw_signals.pkl' #Change to 'input/Data_Raw_Signals.pkl' when not in Colab
spectrogram_dir = 'input/Data_Spectrograms.pkl'

print("Loading Data File..\n")
with open(raw_signal_dir, 'rb') as raw:
    raw_signals, classes = pkl.load(raw)

# with open(spectrogram_dir, 'rb') as specs:
#     spectrogram_data = pkl.load(specs)


# print(type(raw_signal_data))
# print(len(raw_signal_data))
# # print(raw_signal_data.shape)
# print(type(raw_signal_data[0]))
# print(type(raw_signal_data[1]))
# print(raw_signal_data[1].shape)
# print(len(raw_signal_data[1]))
# print(raw_signal_data[0].shape)

# for i in range (125,130):
#     print(raw_signal_data[1][i])
#     print(raw_signal_data[0][i].shape)
#     print(raw_signal_data[0][i])

print(type(raw_signals))
print(len(raw_signals))
print(type(classes))
print(len(classes))
print(raw_signals.shape)
print(classes.shape)

for i in range(125,130):
  print(raw_signals[i])
  print(raw_signals[i].shape)
  print(classes[i])

print("Setting up variables..\n")
# classes = raw_signal_data[1]
# raw_signals = raw_signal_data[0]

le = LabelEncoder()

print("Creating labels...\n")
# labels = le.fit_transform(classes)
labels = np_utils.to_categorical(classes, 
num_classes=6)

print("Splitting data...\n")

# Splitting data from train into train and test data
x_train, x_val, y_train, y_val = train_test_split(
    raw_signals,
labels,stratify = labels, train_size = 0.7, test_size = 0.3,
random_state=777,shuffle=True)

print("Checking train/val data shape...\n")
print(x_train.shape)
print(x_val.shape)


Loading Data File..

<class 'numpy.ndarray'>
15375
<class 'numpy.ndarray'>
15375
(15375, 2, 3000)
(15375,)
[[17.02984801 10.07141549  6.77531588 ...  6.40908259 12.63504853
   8.60648233]
 [-5.676616   -2.74674968 -2.0142831  ... -2.38051639 -0.18311665
   5.31038271]]
(2, 3000)
2
[[13.36751511 26.55191357 31.31294635 ... -5.676616   -5.676616
  -2.38051639]
 [ 5.676616    4.94414942  4.94414942 ...  4.21168284  7.87401575
   6.40908259]]
(2, 3000)
2
[[ -0.54934994  -0.54934994 -10.07141549 ...   4.21168284  -0.91558323
    2.74674968]
 [  4.57791613  10.43764878   9.7051822  ...  -1.64804981   0.54934994
   -1.64804981]]
(2, 3000)
2
[[  3.11298297  -3.11298297  -6.0428493  ... -22.15711408 -22.88958066
  -24.35451383]
 [ -1.28181652   2.0142831    0.54934994 ... -16.66361472 -16.66361472
  -18.49478118]]
(2, 3000)
2
[[-33.1441128  -40.8350119  -48.15967771 ...  13.36751511   3.84544955
    5.31038271]
 [-19.22724776 -21.4246475  -24.35451383 ...  -6.0428493   -5.31038271
   -3.1129829

**Building the model**

In [0]:
print("Building model..\n")
# Building 1D Convolution model
shape = x_train.shape
inputs = Input(shape=(shape[1], shape[2]))
print(shape[1], shape[2])

##
# 1D Convolution + LSTM Modelmodel working with data 80:20, batch=32
print("#############################################")
print("Sequential..")
model = Sequential()
print("First Conv layer..")
model.add(Conv1D(32,4, activation='relu', kernel_regularizer=l2(0.01), 
                 padding='same', input_shape=(shape[1], shape[2])))
print("Second Conv layer..")
model.add(Conv1D(32,4, activation='relu', kernel_regularizer=l2(0.01), 
                 padding='same'))
print("Third Conv layer..")
model.add(Conv1D(64,4, activation='relu', kernel_regularizer=l2(0.01), 
                 padding='same'))
print("Fourth Conv layer..")
model.add(Conv1D(64,4, activation='relu', kernel_regularizer=l2(0.01), 
                 padding='same'))
print("Max Pool (1D) layer..")
model.add(MaxPooling1D(2))

print("First LSTM layer..")
model.add(LSTM(512, return_sequences=True, input_shape=(shape[1], shape[2])))
print("Second LSTM layer..")
model.add(LSTM(128, return_sequences=True))
print("Third LSTM layer..")
model.add(LSTM(65, return_sequences=True))
# print("Max Pool (1D) layer..")
# model.add(MaxPooling1D(2))
print("Dropout layer..")
model.add(Dropout(0.05))
print("Flatten layer..")
model.add(Flatten())
print("Dense layer..")
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(6, activation='softmax'))
model.summary()

print("Compiling model...")
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

print("Generating model name..")
#Setting the model name
hdf_count = len(glob.glob1('/models',"*.hdf5"))
model_name = "model(" + str(hdf_count + 1) + ").hdf5"
print("Model: {}".format(model_name))

print("Setting up training checkpoints...")
# Setting up easy stoping and model checkpoints
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,
 patience=10, min_delta=0.0001) 
mc = ModelCheckpoint(model_name, monitor='val_acc', 
verbose=1, save_best_only=True, mode='max')



Building model..

2 3000
#############################################
Sequential..
First Conv layer..
Second Conv layer..
Third Conv layer..
Fourth Conv layer..
Max Pool (1D) layer..
First LSTM layer..
Second LSTM layer..
Third LSTM layer..
Dropout layer..
Flatten layer..
Dense layer..
Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_34 (Conv1D)           (None, 2, 32)             384032    
_________________________________________________________________
conv1d_35 (Conv1D)           (None, 2, 32)             4128      
_________________________________________________________________
conv1d_36 (Conv1D)           (None, 2, 64)             8256      
_________________________________________________________________
conv1d_37 (Conv1D)           (None, 2, 64)             16448     
_________________________________________________________________
max_pooling1d_15 (MaxPooling 

**Fitting/Training Model**

In [0]:
print("Fitting/Training the model...")
# Fit the model to train

history=model.fit(x_train, y_train ,epochs=100, callbacks=[es,mc], 
batch_size=32, validation_data=(x_val,y_val), verbose=1)

model.evaluate(x_val, y_val)

Fitting/Training the model...
Train on 10762 samples, validate on 4613 samples
Epoch 1/100
10720/10762 [============================>.] - ETA: 0s - loss: 1.4001 - acc: 0.5053
Epoch 00001: val_acc did not improve from 0.45480
10762/10762 [==============================] - 11s 1ms/sample - loss: 1.3995 - acc: 0.5058 - val_loss: 1.5274 - val_acc: 0.4466
Epoch 2/100
10720/10762 [============================>.] - ETA: 0s - loss: 1.4070 - acc: 0.5074
Epoch 00002: val_acc did not improve from 0.45480
10762/10762 [==============================] - 11s 1ms/sample - loss: 1.4072 - acc: 0.5072 - val_loss: 1.5309 - val_acc: 0.4416
Epoch 3/100
10720/10762 [============================>.] - ETA: 0s - loss: 1.4064 - acc: 0.5060
Epoch 00003: val_acc did not improve from 0.45480
10762/10762 [==============================] - 11s 1ms/sample - loss: 1.4053 - acc: 0.5061 - val_loss: 1.5543 - val_acc: 0.4470
Epoch 4/100
10720/10762 [============================>.] - ETA: 0s - loss: 1.4016 - acc: 0.5047
Epo

[1.5526927039754022, 0.45089963]